<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [80]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [81]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [82]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [83]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [111]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [112]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.508299,-0.999270,-0.067683,0.159898,79.124446,3.442908,-2.267219,0.544772,3.442908
1,0.852637,-0.027526,-0.426537,-4.256863,-21.404855,20.598244,-1.114765,0.992078,20.598244
2,-0.820932,-0.779940,0.850433,1.189984,-88.790462,0.385730,-0.707766,0.613380,0.385730
3,-0.499362,0.941968,-0.403360,3.271192,-34.148015,18.315566,-0.111147,0.713422,18.315566
4,0.978983,0.481421,-0.594755,2.632389,52.205667,-0.264776,0.530335,0.511559,-0.264776
...,...,...,...,...,...,...,...,...,...
95,-0.981610,-0.846257,-0.925827,3707.576816,24.935786,1.533067,-0.000285,0.999644,1.533067
96,-0.939915,0.003184,-0.821339,-7.617717,70.351137,0.555648,2.119895,0.726288,0.555648
97,0.521415,0.691156,0.999915,37.003909,101.689221,5.434026,0.012239,0.866444,5.434026
98,-0.354437,-0.999009,-0.882800,5.356379,-10.272422,0.208683,-0.039414,0.455161,0.208683


In [113]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.978440,0.360269,-0.771925,4.467332,-39.396481,4.042130,-0.360944,0.885246,4.042130
1,-0.969844,-0.242213,-0.741654,-0.051492,-90.067401,-0.724983,37.689130,0.814323,-0.724983
2,-0.854070,-0.210979,-0.513740,11.732353,-34.886986,4.186231,-0.146466,0.929916,4.186231
3,-0.796320,0.408583,-0.445287,0.455390,28.245694,0.812148,-2.441228,0.851919,0.812148
4,-0.434642,0.510286,-0.886204,3.669684,-54.189215,14.661458,-0.109311,0.930419,14.661458
...,...,...,...,...,...,...,...,...,...
95,-0.514997,-0.911968,-0.880467,2.080503,15.757934,-2.856157,-0.186015,0.530637,-2.856157
96,0.947069,0.999650,-0.515281,15.791026,-94.823226,0.321157,0.058377,0.561841,0.321157
97,-0.975833,0.516020,-0.661692,11011.912733,66.134609,1.851467,-0.000122,0.980240,1.851467
98,-0.972949,0.994451,0.211606,1749.331060,-28.169485,-0.003369,-0.000550,0.058042,-0.003369


##1. Линейная форма функции пересчёта

In [87]:
vector_ones = [1] * elem

###Для функции $x_1[t+1]$

In [115]:
#x3, v1, v2
x1_linearFunc = model_9[['x3(9)', 'v1(9)', 'v2(9)']]
x1_linearFunc['x1(10)'] = model_10['x1(10)']
x1_linearFunc.insert(0, "optional_col", vector_ones)

A1_linear = np.array(x1_linearFunc)
#A1_psev = BenIsrael(A1_linear[:, 0:(len(A1_linear[0]) - 1)])
X1_linear = solvingSLAE(A1_linear[:, 0:(len(A1_linear[0]) - 1)], np.matrix(A1_linear[:, len(A1_linear[0]) - 1]).T)
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))

           X1
a1  -0.044574
c13 -0.462642
b11  0.000006
b12  0.000249


<ipython-input-115-c73363b87a45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['x1(10)'] = model_10['x1(10)']


In [116]:
X1_estimate_linear = []
X1_variance_linear = []
X1_error_linear = []
for i in range(len(x1_linearFunc)):
  X1_estimate_linear.append(X1_linear[0,0] + X1_linear[1,0] * np.array(x1_linearFunc)[i,1]
                            + X1_linear[2,0] * np.array(x1_linearFunc)[i,2] + X1_linear[3,0] * np.array(x1_linearFunc)[i,3])

  X1_variance_linear.append((1.0 / len(x1_linearFunc)) * (np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])**2)

  X1_error_linear.append((1.0 / len(x1_linearFunc)) * abs((np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])
                            / np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1]) * 100)

x1_linearFunc['X1*'] = X1_estimate_linear
x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
x1_linearFunc['|(X1 - X1*)| / |X1|, %'] = X1_error_linear
x1_linearFunc

<ipython-input-116-65ca3a007bdb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['X1*'] = X1_estimate_linear
<ipython-input-116-65ca3a007bdb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
<ipython-input-116-65ca3a007bdb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,x3(9),v1(9),v2(9),x1(10),X1*,(X1 - X1*)^2,"|(X1 - X1*)| / |X1|, %"
0,1,-0.067683,0.159898,79.124446,-0.978440,0.006470,0.009700,1.006612
1,1,-0.426537,-4.256863,-21.404855,-0.969844,0.147396,0.012482,1.151980
2,1,0.850433,1.189984,-88.790462,-0.854070,-0.460152,0.001552,0.461225
3,1,-0.403360,3.271192,-34.148015,-0.796320,0.133543,0.008646,1.167700
4,1,-0.594755,2.632389,52.205667,-0.434642,0.243618,0.004600,1.560503
...,...,...,...,...,...,...,...,...
95,1,-0.925827,3707.576816,24.935786,-0.514997,0.412869,0.008609,1.801692
96,1,-0.821339,-7.617717,70.351137,0.947069,0.352907,0.003530,0.627370
97,1,0.999915,37.003909,101.689221,-0.975833,-0.481592,0.002443,0.506482
98,1,-0.882800,5.356379,-10.272422,-0.972949,0.361318,0.017803,1.371364


In [117]:
print("E = " + str(sum(x1_linearFunc['(X1 - X1*)^2'])))
print("A = " + str(sum(x1_linearFunc['|(X1 - X1*)| / |X1|, %'])))

E = 0.3732973084830981
A = 172.76824504367903


###Для функции $x_2[t+1]$

In [118]:
#x2, x3, v1
x2_linearFunc = model_9[['x2(9)', 'x3(9)', 'v1(9)']]
x2_linearFunc['x2(10)'] = model_10['x2(10)']
x2_linearFunc.insert(0, "optional_col", vector_ones)

A2_linear = np.array(x2_linearFunc)
#A2_psev = BenIsrael(A2_linear[:, 0:(len(A2_linear[0]) - 1)])
X2_linear = solvingSLAE(A2_linear[:, 0:(len(A2_linear[0]) - 1)], np.matrix(A2_linear[:, len(A2_linear[0]) - 1]).T)
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))

           X2
a2  -0.049346
c22  0.130301
c23 -0.252204
b21 -0.000037


<ipython-input-118-d5e5fbe5c352>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['x2(10)'] = model_10['x2(10)']


In [119]:
X2_estimate_linear = []
X2_variance_linear = []
X2_error_linear = []
for i in range(len(x2_linearFunc)):
  X2_estimate_linear.append(X2_linear[0,0] + X2_linear[1,0] * np.array(x2_linearFunc)[i,1]
                            + X2_linear[2,0] * np.array(x2_linearFunc)[i,2] + X2_linear[3,0] * np.array(x2_linearFunc)[i,3])

  X2_variance_linear.append((1.0 / len(x2_linearFunc)) * (np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])**2)

  X2_error_linear.append((1.0 / len(x2_linearFunc)) * abs((np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1] - X2_estimate_linear[i])
                            / np.array(x2_linearFunc)[i, len(np.array(x2_linearFunc)[0]) - 1]) * 100)

x2_linearFunc['X2*'] = X2_estimate_linear
x2_linearFunc['(X2 - X2*)^2'] = X2_variance_linear
x2_linearFunc['|(X2 - X2*)| / |X2|, %'] = X2_error_linear
x2_linearFunc

<ipython-input-119-75044911546d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['X2*'] = X2_estimate_linear
<ipython-input-119-75044911546d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linearFunc['(X2 - X2*)^2'] = X2_variance_linear
<ipython-input-119-75044911546d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,x2(9),x3(9),v1(9),x2(10),X2*,(X2 - X2*)^2,"|(X2 - X2*)| / |X2|, %"
0,1,-0.999270,-0.067683,0.159898,0.360269,-0.162488,0.002733,1.451017
1,1,-0.027526,-0.426537,-4.256863,-0.242213,0.054799,0.000882,1.226244
2,1,-0.779940,0.850433,1.189984,-0.210979,-0.365500,0.000239,0.732403
3,1,0.941968,-0.403360,3.271192,0.408583,0.175000,0.000546,0.571690
4,1,0.481421,-0.594755,2.632389,0.510286,0.163285,0.001204,0.680012
...,...,...,...,...,...,...,...,...
95,1,-0.846257,-0.925827,3707.576816,-0.911968,-0.063552,0.007198,0.930313
96,1,0.003184,-0.821339,-7.617717,0.999650,0.158496,0.007075,0.841448
97,1,0.691156,0.999915,37.003909,0.516020,-0.212843,0.005312,1.412471
98,1,-0.999009,-0.882800,5.356379,0.994451,0.042930,0.009054,0.956831


In [120]:
print("E = " + str(sum(x2_linearFunc['(X2 - X2*)^2'])))
print("A = " + str(sum(x2_linearFunc['|(X2 - X2*)| / |X2|, %'])))

E = 0.43521694247147674
A = 122.56914270528941


###Для функции $x_3[t+1]$

In [121]:
#x3, v1, v3
x3_linearFunc = model_9[['x3(9)', 'v1(9)', 'v3(9)']]
x3_linearFunc['x3(10)'] = model_10['x3(10)']
x3_linearFunc.insert(0, "optional_col", vector_ones)

A3_linear = np.array(x3_linearFunc)
#A3_psev = BenIsrael(A3_linear[:, 0:(len(A3_linear[0]) - 1)])
X3_linear = solvingSLAE(A3_linear[:, 0:(len(A3_linear[0]) - 1)], np.matrix(A3_linear[:, len(A3_linear[0]) - 1]).T)
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))

           X3
a3  -0.178441
c33 -0.060768
b31  0.000030
b33 -0.009462


<ipython-input-121-442e4b3ba6ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['x3(10)'] = model_10['x3(10)']


In [122]:
X3_estimate_linear = []
X3_variance_linear = []
X3_error_linear = []
for i in range(len(x3_linearFunc)):
  X3_estimate_linear.append(X3_linear[0,0] + X3_linear[1,0] * np.array(x3_linearFunc)[i,1]
                            + X3_linear[2,0] * np.array(x3_linearFunc)[i,2] + X3_linear[3,0] * np.array(x3_linearFunc)[i,3])

  X3_variance_linear.append((1.0 / len(x3_linearFunc)) * (np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])**2)

  X3_error_linear.append((1.0 / len(x3_linearFunc)) * abs((np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1] - X3_estimate_linear[i])
                            / np.array(x3_linearFunc)[i, len(np.array(x3_linearFunc)[0]) - 1]) * 100)

x3_linearFunc['X3*'] = X3_estimate_linear
x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
x3_linearFunc['|(X3 - X3*)| / |X3|, %'] = X3_error_linear
x3_linearFunc

<ipython-input-122-0f73eb653e66>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['X3*'] = X3_estimate_linear
<ipython-input-122-0f73eb653e66>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linearFunc['(X3 - X3*)^2'] = X3_variance_linear
<ipython-input-122-0f73eb653e66>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,x3(9),v1(9),v3(9),x3(10),X3*,(X3 - X3*)^2,"|(X3 - X3*)| / |X3|, %"
0,1,-0.067683,0.159898,3.442908,-0.771925,-0.206898,0.003193,0.731971
1,1,-0.426537,-4.256863,20.598244,-0.741654,-0.347541,0.001553,0.531397
2,1,0.850433,1.189984,0.385730,-0.513740,-0.233733,0.000784,0.545036
3,1,-0.403360,3.271192,18.315566,-0.445287,-0.327123,0.000140,0.265366
4,1,-0.594755,2.632389,-0.264776,-0.886204,-0.139714,0.005572,0.842346
...,...,...,...,...,...,...,...,...
95,1,-0.925827,3707.576816,1.533067,-0.880467,-0.023911,0.007337,0.972843
96,1,-0.821339,-7.617717,0.555648,-0.515281,-0.134019,0.001454,0.739911
97,1,0.999915,37.003909,5.434026,-0.661692,-0.289492,0.001385,0.562498
98,1,-0.882800,5.356379,0.208683,0.211606,-0.126607,0.001144,1.598312


In [123]:
print("E = " + str(sum(x3_linearFunc['(X3 - X3*)^2'])))
print("A = " + str(sum(x3_linearFunc['|(X3 - X3*)| / |X3|, %'])))

E = 0.4712850777998747
A = 109.41410238113747


###Для функции $y_1[t]$

In [124]:
#x1, x2, v1
y1_linearFunc = model_9[['x1(9)', 'x2(9)', 'v1(9)']]
y1_linearFunc['y1(9)'] = model_9['y1(9)']
y1_linearFunc.insert(0, "optional_col", vector_ones)

A1y_linear = np.array(y1_linearFunc)
#A1y_psev = BenIsrael(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)])
Y1_linear = solvingSLAE(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)], np.matrix(A1y_linear[:, len(A1y_linear[0]) - 1]).T)
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))

           Y1
a1  -0.306699
c11 -0.276708
c12  0.608735
b11  0.000008


<ipython-input-124-62570c5c1727>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['y1(9)'] = model_9['y1(9)']


In [125]:
Y1_estimate_linear = []
Y1_variance_linear = []
Y1_error_linear = []
for i in range(len(y1_linearFunc)):
  Y1_estimate_linear.append(Y1_linear[0,0] + Y1_linear[1,0] * np.array(y1_linearFunc)[i,1]
                            + Y1_linear[2,0] * np.array(y1_linearFunc)[i,2] + Y1_linear[3,0] * np.array(y1_linearFunc)[i,3])

  Y1_variance_linear.append((1.0 / len(y1_linearFunc)) * (np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])**2)

  Y1_error_linear.append((1.0 / len(y1_linearFunc)) * abs((np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1] - Y1_estimate_linear[i])
                            / np.array(y1_linearFunc)[i, len(np.array(y1_linearFunc)[0]) - 1]) * 100)

y1_linearFunc['Y1*'] = Y1_estimate_linear
y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
y1_linearFunc['|(Y1 - Y1*)| / |Y1|, %'] = Y1_error_linear
y1_linearFunc

<ipython-input-125-97159da812be>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['Y1*'] = Y1_estimate_linear
<ipython-input-125-97159da812be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linearFunc['(Y1 - Y1*)^2'] = Y1_variance_linear
<ipython-input-125-97159da812be>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,x1(9),x2(9),v1(9),y1(9),Y1*,(Y1 - Y1*)^2,"|(Y1 - Y1*)| / |Y1|, %"
0,1,-0.508299,-0.999270,0.159898,-2.267219,-0.774337,0.022287,0.658464
1,1,0.852637,-0.027526,-4.256863,-1.114765,-0.559422,0.003084,0.498170
2,1,-0.820932,-0.779940,1.189984,-0.707766,-0.554307,0.000235,0.216822
3,1,-0.499362,0.941968,3.271192,-0.111147,0.404915,0.002663,4.643045
4,1,0.978983,0.481421,2.632389,0.530335,-0.284511,0.006640,1.536474
...,...,...,...,...,...,...,...,...
95,1,-0.981610,-0.846257,3707.576816,-0.000285,-0.518913,0.002690,1818.815054
96,1,-0.939915,0.003184,-7.617717,2.119895,-0.044743,0.046857,1.021106
97,1,0.521415,0.691156,37.003909,0.012239,-0.029935,0.000018,3.445900
98,1,-0.354437,-0.999009,5.356379,-0.039414,-0.816709,0.006042,19.721219


In [126]:
print("E = " + str(sum(y1_linearFunc['(Y1 - Y1*)^2'])))
print("A = " + str(sum(y1_linearFunc['|(Y1 - Y1*)| / |Y1|, %'])))

E = 81.36252000645362
A = 35857.44189149783


###Для функции $y_2[t]$

In [127]:
#v3
y2_linearFunc = model_9[['v3(9)']]
y2_linearFunc['y2(9)'] = model_9['y2(9)']
y2_linearFunc.insert(0, "optional_col", vector_ones)

A2y_linear = np.array(y2_linearFunc)
#A2y_psev = BenIsrael(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)])
Y2_linear = solvingSLAE(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)], np.matrix(A2y_linear[:, len(A2y_linear[0]) - 1]).T)
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))

           Y2
a2   0.742650
b23  0.004967


<ipython-input-127-993de31ed838>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['y2(9)'] = model_9['y2(9)']


In [128]:
Y2_estimate_linear = []
Y2_variance_linear = []
Y2_error_linear = []
for i in range(len(y2_linearFunc)):
  Y2_estimate_linear.append(Y2_linear[0,0] + Y2_linear[1,0] * np.array(y2_linearFunc)[i,1])

  Y2_variance_linear.append((1.0 / len(y2_linearFunc)) * (np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])**2)

  Y2_error_linear.append((1.0 / len(y2_linearFunc)) * abs((np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1] - Y2_estimate_linear[i])
                            / np.array(y2_linearFunc)[i, len(np.array(y2_linearFunc)[0]) - 1]) * 100)

y2_linearFunc['Y2*'] = Y2_estimate_linear
y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
y2_linearFunc['|(Y2 - Y2*)| / |Y2|, %'] = Y2_error_linear
y2_linearFunc

<ipython-input-128-7cd0bf4dc686>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['Y2*'] = Y2_estimate_linear
<ipython-input-128-7cd0bf4dc686>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linearFunc['(Y2 - Y2*)^2'] = Y2_variance_linear
<ipython-input-128-7cd0bf4dc686>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,v3(9),y2(9),Y2*,(Y2 - Y2*)^2,"|(Y2 - Y2*)| / |Y2|, %"
0,1,3.442908,0.544772,0.759752,0.000462,0.394624
1,1,20.598244,0.992078,0.844966,0.000216,0.148286
2,1,0.385730,0.613380,0.744566,0.000172,0.213873
3,1,18.315566,0.713422,0.833628,0.000144,0.168491
4,1,-0.264776,0.511559,0.741335,0.000528,0.449167
...,...,...,...,...,...,...
95,1,1.533067,0.999644,0.750265,0.000622,0.249468
96,1,0.555648,0.726288,0.745410,0.000004,0.026329
97,1,5.434026,0.866444,0.769642,0.000094,0.111723
98,1,0.208683,0.455161,0.743687,0.000832,0.633897


In [129]:
print("E = " + str(sum(y2_linearFunc['(Y2 - Y2*)^2'])))
print("A = " + str(sum(y2_linearFunc['|(Y2 - Y2*)| / |Y2|, %'])))

E = 0.0553144862215651
A = 39.232625415361426


###Для функции $y_3[t]$

In [130]:
#v1, v3
y3_linearFunc = model_9[['v1(9)', 'v3(9)']]
y3_linearFunc['y3(9)'] = model_9['y3(9)']
y3_linearFunc.insert(0, "optional_col", vector_ones)

A3y_linear = np.array(y3_linearFunc)
#A3y_psev = BenIsrael(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)])
Y3_linear = solvingSLAE(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)], np.matrix(A3y_linear[:, len(A3y_linear[0]) - 1]).T)
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))

               Y3
a3   9.013659e-08
b31 -3.271126e-12
b33  1.000000e+00


<ipython-input-130-abfc1d45964a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['y3(9)'] = model_9['y3(9)']


In [131]:
Y3_estimate_linear = []
Y3_variance_linear = []
Y3_error_linear = []
for i in range(len(y3_linearFunc)):
  Y3_estimate_linear.append(Y3_linear[0,0] + Y3_linear[1,0] * np.array(y3_linearFunc)[i,1]
                            + Y3_linear[2,0] * np.array(y3_linearFunc)[i,2])

  Y3_variance_linear.append((1.0 / len(y3_linearFunc)) * (np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])**2)

  Y3_error_linear.append((1.0 / len(y3_linearFunc)) * abs((np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1] - Y3_estimate_linear[i])
                            / np.array(y3_linearFunc)[i, len(np.array(y3_linearFunc)[0]) - 1]) * 100)

y3_linearFunc['Y3*'] = Y3_estimate_linear
y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
y3_linearFunc['|(Y3 - Y3*)| / |Y3|, %'] = Y3_error_linear
y3_linearFunc

<ipython-input-131-e73931a3ed05>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['Y3*'] = Y3_estimate_linear
<ipython-input-131-e73931a3ed05>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linearFunc['(Y3 - Y3*)^2'] = Y3_variance_linear
<ipython-input-131-e73931a3ed05>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,optional_col,v1(9),v3(9),y3(9),Y3*,(Y3 - Y3*)^2,"|(Y3 - Y3*)| / |Y3|, %"
0,1,0.159898,3.442908,3.442908,3.442908,3.623457e-17,1.748381e-08
1,1,-4.256863,20.598244,20.598244,20.598244,7.917458e-17,4.319791e-09
2,1,1.189984,0.385730,0.385730,0.385730,7.530462e-17,2.249716e-07
3,1,3.271192,18.315566,18.315566,18.315566,4.782226e-17,3.775676e-09
4,1,2.632389,-0.264776,-0.264776,-0.264776,8.543412e-17,3.490898e-07
...,...,...,...,...,...,...,...
95,1,3707.576816,1.533067,1.533067,1.533067,4.183047e-17,4.218765e-08
96,1,-7.617717,0.555648,0.555648,0.555648,7.281102e-17,1.535674e-07
97,1,37.003909,5.434026,5.434026,5.434026,1.828338e-17,7.868763e-09
98,1,5.356379,0.208683,0.208683,0.208683,7.797645e-17,4.231501e-07


In [132]:
print("E = " + str(sum(y3_linearFunc['(Y3 - Y3*)^2'])))
print("A = " + str(sum(y3_linearFunc['|(Y3 - Y3*)| / |Y3|, %'])))

E = 5.083518727085377e-15
A = 1.43829577497938e-05
